#Global Variables definition:

In [1]:
import pickle

PLEASE MAKE SURE TO EDIT THE PATHS TO THE DATASETS AND THE PARAMETERS SUCH AS THE CNN BACKBONE 'base' ACCORDING TO YOUR CASE SCENARIO

In [2]:
params = {'base':'resnet',
        'dim':(224,224),
        'db_path':'./test/test/',
        'num_frames':8,
        'num_patches':1
        }

In [3]:
list_IDs_path='./ids_test.pickle'
pickle_in = open(list_IDs_path,'rb')
ids= pickle.load(pickle_in)
pickle_in.close()

In [4]:
out='Features/resnet50/test'

#CNN backbone 

In [5]:
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.applications.vgg19 import VGG19
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.applications import DenseNet121
from tensorflow.keras.applications import InceptionV3

2023-03-15 15:32:10.989513: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-03-15 15:32:11.678557: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda/lib64
2023-03-15 15:32:11.678617: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda/lib64
2023-03-15 15:32:11.678622: W tensorflow/compiler/tf2tensorrt/utils/py_utils.c

In [6]:
def Base_Model(base,weights='imagenet', include_top=False, input_shape=(299, 299, 3)):
    if(base=='resnet50'):
        return ResNet50(weights=weights, include_top=include_top, input_shape=input_shape)
    if(base=='vgg16'):
        return VGG16(weights=weights, include_top=include_top, input_shape=input_shape)
    if(base=='vgg19'):
        return VGG19(weights=weights, include_top=include_top, input_shape=input_shape)
    if(base=='densenet121'):
        return DenseNet121(weights=weights, include_top=include_top, input_shape=input_shape)
    if(base=='inceptionv3'):
        return InceptionV3(weights=weights, include_top=include_top, input_shape=input_shape)

#Data generator

In [7]:
import cv2
import pickle
import slidingwindow as sw
from tensorflow import keras
import numpy as np
import copy

In [8]:
import tensorflow as tf

In [9]:
class DataGenerator(keras.utils.Sequence):
    def __init__(self,dim=(224,224),n_channels=3,n_output=1,base='resnet',
                db_path='KoNViD_1k_videos/',threshold=0.5,
                ids=[],num_frames=8,num_patches=6,max_p=30):
        'Initialization'
        self.max_p=max_p
        self.num_patches=num_patches
        self.num_frames=num_frames
        self.batch_size= 1
        self.dim = dim
        self.n_channels = n_channels
        self.n_output = n_output
        self.base=base
        self.db_path=db_path
        self.ids_path=ids
        self.list_IDs_temp=[]
        self.list_IDs=ids
        self.threshold=threshold
                

        vidcap = cv2.VideoCapture(os.path.join(self.db_path,id))
        self.N = int(vidcap.get(cv2.CAP_PROP_FRAME_COUNT))
        self.fps = int(vidcap.get(cv2.CAP_PROP_FPS))
        print('s: ',self.N/self.fps)
        print('fps: ',self.fps)

        success,image = vidcap.read()
        ov = 0
        windows = sw.generate(image, sw.DimOrder.HeightWidthChannel,self.dim[0],ov)
        
        while len(windows) < self.num_patches:
            ov =ov+ 0.1
            windows = sw.generate(image, sw.DimOrder.HeightWidthChannel,self.dim[0],ov)
            if ov > self.threshold:
              break;
        self.ov=ov

        if len(windows) > max_p:
          self.p=max_p
        else:
          self.p= len(windows)




        self.on_epoch_end()

    def __len__(self):
        'Denotes the number of batches per epoch'
        return int(np.floor(len(self.list_IDs)/ self.batch_size))

    def __getitem__(self, index):
        'Generate one batch of data'
        # Generate indexes of the batch
        indexes = self.indexes[index*self.batch_size:(index+1)*self.batch_size]
        # Find list of IDs
        self.list_IDs_temp = [self.list_IDs[k] for k in indexes]
        # Generate data
        X = self.__data_generation(self.list_IDs_temp)
        return X
    
    def on_epoch_end(self):
        'Updates indexes after each epoch'
        self.indexes = np.arange(len(self.list_IDs))
        

    def __data_generation(self, list_IDs_temp):
      for i, ID in enumerate(list_IDs_temp):
        n=self.dim[0]
    
        images=[]
        vidcap = cv2.VideoCapture(os.path.join(self.db_path,ID))
        success,image = vidcap.read()
       
       
       
       
        count = 0;
        while success:
          if count % self.fps ==0:
            images.append(cv2.cvtColor(image,cv2.COLOR_BGR2RGB))
          count += 1
          if (count+1) == self.num_frames:
            images.append(cv2.cvtColor(image,cv2.COLOR_BGR2RGB))
          success,image = vidcap.read()
       
        

        X = np.empty((self.num_frames,self.p,*self.dim, self.n_channels))
            
        for k in range(self.num_frames):

            image=images[k]

            windows = sw.generate(image, sw.DimOrder.HeightWidthChannel,n,self.ov)
                
            for l in range(self.p):  
                
                if self.p < self.max_p:    
                    subset = image[windows[l].indices()]
                else:
                    subset=image[windows[int(len(windows)/self.p*l)].indices()]
                    
                if self.base=='vgg16':
                      subset=keras.applications.vgg16.preprocess_input(subset)
                elif self.base=='inceptionv3':
                      subset=keras.applications.inception_v3.preprocess_input(subset)
                elif self.base=='resnet':
                      subset=keras.applications.resnet.preprocess_input(subset)
                elif self.base=='densenet121':
                      subset=keras.applications.densenet.preprocess_input(subset)
                else:
                      print("No preprocessing..")
                X[k,l,:,:,:] =np.array(subset) 

                             
      return X

#Features extraction:

In [10]:
from tensorflow.keras.layers import MaxPooling2D,Input,GlobalMaxPooling2D,GlobalAveragePooling2D,AveragePooling2D
from tensorflow.keras import layers

In [11]:
base_model =  Base_Model('resnet50',weights='imagenet', include_top=False, input_shape=(224,224, 3))

x=base_model.layers[-1].output
x=GlobalAveragePooling2D()(x)
model=keras.Model(inputs=base_model.layers[0].output,outputs=x)

2023-03-15 15:32:14.460533: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-03-15 15:32:15.079532: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1613] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 5077 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 2080 Ti, pci bus id: 0000:91:00.0, compute capability: 7.5


In [12]:
import os

In [13]:
import time

In [14]:
out

'Features/resnet50/test'

In [15]:
#model definition
input=Input(shape=(None,224,224,3))
output= layers.TimeDistributed(model)(input)
model_cnn=keras.Model(inputs=input,outputs=output)   
t=[]
for id in ids:
 #try:
 #   np.load(os.path.join(out,id+'.npy'))
 #except:
        t1 = time.time()
        generator = DataGenerator(ids=[id],**params)
        feature= model_cnn.predict_generator(generator=generator)
        t2 = time.time()
        t.append(t2-t1)
        np.save(os.path.join(out,id+'.npy'),feature)

s:  10.521739130434783
fps:  23


/tmp/ipykernel_1075551/1458396577.py:12: UserWarning: `Model.predict_generator` is deprecated and will be removed in a future version. Please use `Model.predict`, which supports generators.
  feature= model_cnn.predict_generator(generator=generator)
2023-03-15 15:32:19.865785: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:428] Loaded cuDNN version 8700


s:  10.521739130434783
fps:  23
s:  10.521739130434783
fps:  23
s:  10.521739130434783
fps:  23
s:  10.521739130434783
fps:  23
s:  10.521739130434783
fps:  23
s:  10.1
fps:  30
s:  10.478260869565217
fps:  23
s:  10.413793103448276
fps:  29
s:  10.413793103448276
fps:  29
s:  10.413793103448276
fps:  29
s:  10.413793103448276
fps:  29
s:  10.413793103448276
fps:  29
s:  10.033333333333333
fps:  30
s:  10.344827586206897
fps:  29
s:  10.413793103448276
fps:  29
s:  10.413793103448276
fps:  29
s:  10.413793103448276
fps:  29
s:  10.413793103448276
fps:  29
s:  10.413793103448276
fps:  29
s:  10.033333333333333
fps:  30
s:  10.344827586206897
fps:  29
s:  10.379310344827585
fps:  29
s:  10.379310344827585
fps:  29
s:  10.379310344827585
fps:  29
s:  10.379310344827585
fps:  29
s:  10.379310344827585
fps:  29
s:  10.379310344827585
fps:  29
s:  10.066666666666666
fps:  30
s:  10.344827586206897
fps:  29
s:  10.066666666666666
fps:  30
s:  10.066666666666666
fps:  30
s:  10.066666666666666

In [16]:
np.mean(t)

1.5258507672034705

In [ ]:
torch.cuda.is_available()

In [ ]:
ls train

train.zip


In [ ]:
!unzip train/train.zip


  inflating: __MACOSX/train/._A0078_01.mp4  
  inflating: train/B0012_03.mp4      
  inflating: __MACOSX/train/._B0012_03.mp4  
  inflating: train/A0076_04.mp4      
  inflating: __MACOSX/train/._A0076_04.mp4  
  inflating: train/C0035_03.mp4      
  inflating: __MACOSX/train/._C0035_03.mp4  
  inflating: train/A0002_08.mp4      
  inflating: train/C0060_02.mp4      
  inflating: __MACOSX/train/._C0060_02.mp4  
  inflating: train/C0011_02.mp4      
  inflating: __MACOSX/train/._C0011_02.mp4  
  inflating: train/B0010_06.mp4      
  inflating: __MACOSX/train/._B0010_06.mp4  
  inflating: train/A0073_08.mp4      
  inflating: __MACOSX/train/._A0073_08.mp4  
  inflating: train/A0074_01.mp4      
  inflating: __MACOSX/train/._A0074_01.mp4  
  inflating: train/C0044_03.mp4      
  inflating: __MACOSX/train/._C0044_03.mp4  
  inflating: train/B0036_02.mp4      
  inflating: __MACOSX/train/._B0036_02.mp4  
  inflating: train/B0038_07.mp4      
  inflating: __MACOSX/train/._B0038_07.mp4  
  i